In [19]:
import numpy as np
from scipy.sparse import csgraph

import sys
sys.path.append('../')
from src import data
from scipy.stats import spearmanr

# https://github.com/aestrivex/bctpy
# dokumentace asi jenom pro matlab? https://sites.google.com/site/bctnet/
# from bct.algorithms import efficiency

from netneurotools import metrics # zjistit, proč tohle nefunguje přes pip 
# (mám to stažené lokálně z Githubu a to pak funguje)

# https://enigma-toolbox.readthedocs.io/en/latest/pages/05.HCP/index.html
# enigmatoolbox obsahuje chybu, lokálně jsem si ji opravila, TODO na GitHub
from enigmatoolbox.datasets import load_sc, load_fc
from nilearn import plotting

# https://neuraldatascience.io/7-eeg/mne_data.html
%matplotlib widget

PLOT=False

!date -I

2024-01-26


In [20]:
# load data
response_probability = np.loadtxt('../data/external/F-TRACT_DKT/probability.txt.gz')
parcells_list = ['ctx-lh-bankssts','ctx-lh-caudalanteriorcingulate','ctx-lh-caudalmiddlefrontal','ctx-lh-cuneus','ctx-lh-entorhinal','ctx-lh-fusiform','ctx-lh-inferiorparietal','ctx-lh-inferiortemporal','ctx-lh-isthmuscingulate','ctx-lh-lateraloccipital','ctx-lh-lateralorbitofrontal','ctx-lh-lingual','ctx-lh-medialorbitofrontal','ctx-lh-middletemporal','ctx-lh-parahippocampal','ctx-lh-paracentral','ctx-lh-parsopercularis','ctx-lh-parsorbitalis','ctx-lh-parstriangularis','ctx-lh-pericalcarine','ctx-lh-postcentral','ctx-lh-posteriorcingulate','ctx-lh-precentral','ctx-lh-precuneus','ctx-lh-rostralanteriorcingulate','ctx-lh-rostralmiddlefrontal','ctx-lh-superiorfrontal','ctx-lh-superiorparietal','ctx-lh-superiortemporal','ctx-lh-supramarginal','ctx-lh-frontalpole','ctx-lh-temporalpole','ctx-lh-transversetemporal','ctx-lh-insula','ctx-rh-bankssts','ctx-rh-caudalanteriorcingulate','ctx-rh-caudalmiddlefrontal','ctx-rh-cuneus','ctx-rh-entorhinal','ctx-rh-fusiform','ctx-rh-inferiorparietal','ctx-rh-inferiortemporal','ctx-rh-isthmuscingulate','ctx-rh-lateraloccipital','ctx-rh-lateralorbitofrontal','ctx-rh-lingual','ctx-rh-medialorbitofrontal','ctx-rh-middletemporal','ctx-rh-parahippocampal','ctx-rh-paracentral','ctx-rh-parsopercularis','ctx-rh-parsorbitalis','ctx-rh-parstriangularis','ctx-rh-pericalcarine','ctx-rh-postcentral','ctx-rh-posteriorcingulate','ctx-rh-precentral','ctx-rh-precuneus','ctx-rh-rostralanteriorcingulate','ctx-rh-rostralmiddlefrontal','ctx-rh-superiorfrontal','ctx-rh-superiorparietal','ctx-rh-superiortemporal','ctx-rh-supramarginal','ctx-rh-frontalpole','ctx-rh-temporalpole','ctx-rh-transversetemporal','ctx-rh-insula']

with open('../data/external/F-TRACT_DKT/DKT.txt', 'r') as f:
    parcell_ids =  [i for i,line in enumerate(f) if line[:-1] in parcells_list]

response_probability = response_probability[np.ix_(parcell_ids, parcell_ids)]

In [46]:
# Load cortico-cortical structural connectivity data
SC, sc_ctx_labels, _, _ = load_sc()
FC, fc_ctx_labels, _, _ = load_fc()

# Plot cortico-cortical connectivity matrices
if PLOT:
    sc_plot = plotting.plot_matrix(SC, figure=(9, 9), labels=sc_ctx_labels, vmax=10, vmin=0, cmap='Blues')  
    fc_plot = plotting.plot_matrix(FC, figure=(9, 9), labels=fc_ctx_labels, vmax=10, vmin=0, cmap='Reds')  

# distance matrix from SC
L = np.where(SC==0.0,0,1/SC)

def coor_with_response_probability(X):
    return spearmanr(X.flatten(),response_probability.flatten(),nan_policy='omit')

/tmp/ipykernel_23769/2541278257.py:11: RuntimeWarning: divide by zero encountered in divide
  L = np.where(SC==0.0,0,1/SC)


SignificanceResult(statistic=0.04545101922729397, pvalue=0.10641779350755716)

In [39]:
coor_with_response_probability(SC)

SignificanceResult(statistic=0.3488356789960631, pvalue=1.89464172494047e-37)

In [40]:
coor_with_response_probability(FC)

SignificanceResult(statistic=0.04545101922729397, pvalue=0.10641779350755716)

In [41]:
# shortest path efficiency
shorest_paths,_ = metrics.distance_wei_floyd(L)
shorest_path_eff = np.divide(1,shorest_paths)

coor_with_response_probability(shorest_path_eff)

/tmp/ipykernel_23769/1652578655.py:3: RuntimeWarning: divide by zero encountered in divide
  shorest_path_eff = np.divide(1,shorest_paths)


SignificanceResult(statistic=0.41326369998828494, pvalue=1.3399665086721805e-51)

In [42]:
com = metrics.communicability_wei(SC)

coor_with_response_probability(com)

SignificanceResult(statistic=0.34072505429401195, pvalue=1.0609201494863236e-35)

In [43]:
SI = metrics.search_information(SC,L)

coor_with_response_probability(SI)

SignificanceResult(statistic=-0.40619054270894844, pvalue=9.54648448432795e-50)

In [44]:
# https://netneurolab.github.io/netneurotools/_modules/netneurotools/metrics.html#diffusion_efficiency
diffusion = metrics.diffusion_efficiency(SC)[1] # tahle funkce určitě žere SC, a ne délky L

coor_with_response_probability(diffusion)

/home/katterrina/fi_muni/0_diplomka/__venv__/lib/python3.10/site-packages/netneurotools/metrics.py:780: RuntimeWarning: divide by zero encountered in divide
  E_diff = np.divide(1, mfpt)


SignificanceResult(statistic=-0.03070366004996593, pvalue=0.2755611381535449)

In [45]:
# tady té funkci možná dávám špatný vstup?
_,_,nav_paths,_,_ = metrics.navigation_wu(L, SC)
nav_eff = np.divide(1,nav_paths)

coor_with_response_probability(nav_eff)

/tmp/ipykernel_23769/2989905826.py:3: RuntimeWarning: divide by zero encountered in divide
  nav_eff = np.divide(1,nav_paths)


SignificanceResult(statistic=0.06815598330241136, pvalue=0.017178279263492316)